In [ ]:
! pip install --upgrade ibis-framework
! pip install pyarrow==<compatible_version>

/bin/bash: -c: line 1: syntax error near unexpected token `newline'
/bin/bash: -c: line 1: ` pip install pyarrow==<compatible_version>'


In [ ]:

!pip install datasets

In [ ]:
from datasets import load_dataset
import torch
import matplotlib.pyplot as plt
from transformers import pipeline

In [ ]:
!pip install wandb

In [ ]:
import wandb
wandb.init(project="FineTune-TextSummarize", name="Fariha")

wandb: Currently logged in as: fariha_shah (k21). Use `wandb login --relogin` to force relogin


In [ ]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge-score py7zr -q


In [ ]:
!pip install accelerate

In [ ]:
import accelerate

print(accelerate.__version__)

0.30.1


In [ ]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [ ]:
!pip install tqdm

In [ ]:
from transformers import pipeline, set_seed

import matplotlib.pyplot as plt
from datasets import load_dataset
import pandas as pd
from datasets import load_dataset, load_metric

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

import nltk

from nltk.tokenize import sent_tokenize

from tqdm import tqdm
import torch

nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
device ="cuda" if  torch.cuda.is_available() else "cpu"
device

'cpu'

In [ ]:
model_id="google/pegasus-xsum"
tokenizer = AutoTokenizer.from_pretrained(model_id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
model_pegasus= AutoModelForSeq2SeqLM.from_pretrained(model_id).to(device)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
#load data
dataset=load_dataset("samsum")
dataset


/usr/local/lib/python3.10/dist-packages/datasets/load.py:1486: FutureWarning: The repository for samsum contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/samsum
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [ ]:
split_lengths= [len(dataset[split])for split in dataset]
split_lengths

[14732, 819, 818]

In [ ]:
#lets check for columns
print(f"Features:{dataset['train'].column_names}")

Features:['id', 'dialogue', 'summary']


In [ ]:
#let's check the example for the same
print("\nDialogue:")

print(dataset['test'][1]["dialogue"])

print("\nsummary:")

print(dataset["test"][1]["summary"])


Dialogue:
Eric: MACHINE!
Rob: That's so gr8!
Eric: I know! And shows how Americans see Russian ;)
Rob: And it's really funny!
Eric: I know! I especially like the train part!
Rob: Hahaha! No one talks to the machine like that!
Eric: Is this his only stand-up?
Rob: Idk. I'll check.
Eric: Sure.
Rob: Turns out no! There are some of his stand-ups on youtube.
Eric: Gr8! I'll watch them now!
Rob: Me too!
Eric: MACHINE!
Rob: MACHINE!
Eric: TTYL?
Rob: Sure :)

summary:
Eric and Rob are going to watch a stand-up on youtube.


In [ ]:
#Evaluating PEGASUS on SamSum

dialogue=dataset['test'][0]['dialogue']
dialogue

"Hannah: Hey, do you have Betty's number?\nAmanda: Lemme check\nHannah: <file_gif>\nAmanda: Sorry, can't find it.\nAmanda: Ask Larry\nAmanda: He called her last time we were at the park together\nHannah: I don't know him well\nHannah: <file_gif>\nAmanda: Don't be shy, he's very nice\nHannah: If you say so..\nHannah: I'd rather you texted him\nAmanda: Just text him 🙂\nHannah: Urgh.. Alright\nHannah: Bye\nAmanda: Bye bye"

In [ ]:
pipe =pipeline('summarization',model=model_id)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
pipe_out=pipe(dialogue)
pipe_out

[{'summary_text': "Amanda: Hey Hannah, do you have Betty's number?"}]

In [ ]:
# As we see the above result is not in the reable mode therefore here we can do some modification for it
print(pipe_out[0]['summary_text'].replace(".<n>",".\n"))

Amanda: Hey Hannah, do you have Betty's number?


In [ ]:
def generate_batch_sized_chunks(list_of_elements, batch_size):
    """
    Split the dataset into smaller batches that we can process simultaneously.
    Yield successive batch-sized chunks from list_of_elements.
    """
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i: i + batch_size]

def calculate_metric_on_test_ds(dataset, metric, model, tokenizer,
                               batch_size=16, device=device,
                               column_text="article",
                               column_summary="highlights"):
    article_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
    target_batches = list(generate_batch_sized_chunks(dataset[column_summary], batch_size))

    for article_batch, target_batch in tqdm(
        zip(article_batches, target_batches), total=len(article_batches)):

        inputs = tokenizer(article_batch, max_length=1024,  truncation=True,
                        padding="max_length", return_tensors="pt")

        summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                         attention_mask=inputs["attention_mask"].to(device),
                         length_penalty=0.8, num_beams=8, max_length=128)
        ''' parameter for length penalty ensures that the model does not generate sequences that are too long. '''

        # Finally, we decode the generated texts,
        # replace the  token, and add the decoded texts with the references to the metric.
        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                                clean_up_tokenization_spaces=True)
               for s in summaries]

        decoded_summaries = [d.replace("", " ") for d in decoded_summaries]


        metric.add_batch(predictions=decoded_summaries, references=target_batch)

    #  Finally compute and return the ROUGE scores.
    score = metric.compute()
    return score

In [ ]:
rouge_metric = load_metric('rouge')


<ipython-input-24-90542a62301a>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge_metric = load_metric('rouge')
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [ ]:
#score = calculate_metric_on_test_ds(dataset_samsum['test'], rouge_metric, model_pegasus, tokenizer, column_text = 'dialogue', column_summary='summary', batch_size=8)
#score = calculate_metric_on_test_ds(dataset['test'], rouge_metric, model_pegasus, tokenizer, column_text='dialogue', column_summary='summary', batch_size=8)
#score = calculate_metric_on_test_ds(dataset['test'], load_metric('rouge'), model_pegasus, tokenizer, column_text='dialogue', column_summary='summary', batch_size=8)


In [ ]:
rouge_names=["rouge1","rouge2","rougeL","rougeLsum"]
#rouge_dict=dict((rn, score[rn].mid.fmeasure)for rn in rouge_names)

#pd.DataFrame(rouge_dict, index=['pegasus'])

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [ ]:
dataset['train']

Dataset({
    features: ['id', 'dialogue', 'summary'],
    num_rows: 14732
})

In [ ]:
dialogue_len=len([tokenizer.encode(s) for s in dataset['train']['dialogue']])
dialogue_len

Token indices sequence length is longer than the specified maximum sequence length for this model (530 > 512). Running this sequence through the model will result in indexing errors


14732

In [ ]:
#converting the data in numerical form
def convert_examples_to_features(example_batch):
    input_encodings=tokenizer(example_batch['dialogue'], max_length=1024,truncation=True)

    with tokenizer.as_target_tokenizer():
        target_encodings= tokenizer(example_batch['summary'],max_length=128,truncation=True)

    return{
        'input_ids':input_encodings['input_ids'],
        'attention_mask':input_encodings['attention_mask'],
        'labels':target_encodings['input_ids']
    }
dataset_pt=dataset.map(convert_examples_to_features, batched=True)

Map:   0%|          | 0/819 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3921: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [ ]:
dataset_pt['train'][0]

{'id': '13818513',
 'dialogue': "Amanda: I baked  cookies. Do you want some?\r\nJerry: Sure!\r\nAmanda: I'll bring you tomorrow :-)",
 'summary': 'Amanda baked cookies and will bring Jerry some tomorrow.',
 'input_ids': [12195,
  151,
  125,
  7091,
  3659,
  107,
  842,
  119,
  245,
  181,
  152,
  10508,
  151,
  7435,
  147,
  12195,
  151,
  125,
  131,
  267,
  650,
  119,
  3469,
  29344,
  1],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 'labels': [12195, 7091, 3659, 111, 138, 650, 10508, 181, 3469, 107, 1]}

In [ ]:
from transformers import DataCollatorForSeq2Seq

seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)

In [ ]:
from transformers import TrainingArguments, Trainer

trainer_args = TrainingArguments(
    output_dir = 'pegasus-samsum', max_steps=200, warmup_steps=20,
    per_device_train_batch_size=1 , per_device_eval_batch_size=1,
    weight_decay = 0.01, logging_steps=10,
    evaluation_strategy ='steps' , eval_steps=50, save_steps=100,
    gradient_accumulation_steps=16,
    report_to="wandb"
)

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
trainer = Trainer(model = model_pegasus, args= trainer_args,
                  tokenizer=tokenizer, data_collator = seq2seq_data_collator,
                  train_dataset= dataset_pt['train'],
                  eval_dataset= dataset_pt['validation']
                 )

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
trainer.train()

In [ ]:
model_pegasus.save_pretrained("pegasus-samsum-model")

In [ ]:
# Read Excel file
df = pd.read_excel("/kaggle/input/pegasus-dataset/Trasncribedtxt.xlsx")


In [ ]:

tokenizer = AutoTokenizer.from_pretrained(model_id)
model_pegasus= AutoModelForSeq2SeqLM.from_pretrained(model_id)


# Function to generate summary
def bart_summarize(input_texts, num_beams, length_penalty, max_length, min_length, no_repeat_ngram_size):
    inputs = tokenizer([input_texts], max_length=1024, return_tensors='pt', truncation=True)
    summary_ids = model_pegasus.generate(inputs['input_ids'], num_beams=num_beams, length_penalty=length_penalty,
                                 max_length=max_length, min_length=min_length, no_repeat_ngram_size=no_repeat_ngram_size)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary


# Process each text entry and generate summaries
for index, row in df.iterrows():
    input_text = row['trasncribed_Text']  # Assuming the column name containing text is 'Text'
    summary = bart_summarize(input_text, num_beams=4, length_penalty=2.0, max_length=250, min_length=100, no_repeat_ngram_size=3)
    df.at[index, 'Summary'] = summary  # Add the summary to a new column 'Summary'

# Write summaries back to Excel file
df.to_excel("output_excel_with_summaries_using_Pegasus_x_large.xlsx", index=False)


In [ ]:
!pip install rouge_score

In [ ]:
import pandas as pd
from rouge_score import rouge_scorer

# Read the Excel file
df = pd.read_excel("output_excel_with_summaries_using_Pegasus_x_large")

# Extract the hypothesis and reference columns from the DataFrame
hyp_col = df['Summary']  # Assuming 'Generated Summary' is the column containing the generated summaries
ref_col = df['trasncribed_Text']      # Assuming 'Reference Text' is the column containing the reference texts

# Initialize RougeScorer object
scorer = rouge_scorer.RougeScorer(['rouge1'])

# Initialize results dictionary
results = {'precision': [], 'recall': [], 'fmeasure': []}

# Loop through each pair of hypothesis and reference
for hyp, ref in zip(hyp_col, ref_col):
    # Compute the ROUGE score
    score = scorer.score(hyp, ref)
    # Separate the measurements
    precision, recall, fmeasure = score['rouge1']
    # Add them to the results dictionary
    results['precision'].append(precision)
    results['recall'].append(recall)
    results['fmeasure'].append(fmeasure)




In [ ]:
print("Precision:", results['precision'])
print("Recall:", results['recall'])
print("F-measure:", results['fmeasure'])

In [ ]:
from google.colab import drive
drive.mount('/content/drive')